<a href="https://colab.research.google.com/github/HussainBadreddeen/AutoML_Thesis/blob/main/new_env_for_TPOT_and_H2o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import drive
drive.mount('/content/drive')
# df_normalized_reviews.to_csv("/content/drive/My Drive/processed_data.csv", index=False, encoding="utf-8-sig") #fixed it utf-8 doesnt wokr although it did before
import pandas as pd
import numpy as np
#load it back
df_normalized_reviews2 = pd.read_csv("/content/drive/My Drive/processed_data.csv") #the nan row comes back somehow


Mounted at /content/drive


In [2]:
print(df_normalized_reviews2[df_normalized_reviews2.isna().any(axis=1)])
empty_or_nan_values = df_normalized_reviews2[df_normalized_reviews2.isna().any(axis=1)]
empty_or_nan_values

df_normalized_reviews2 = df_normalized_reviews2[df_normalized_reviews2['stemmed_text'].notna()]  # remove rows where stemmed_text is NaN
print(df_normalized_reviews2[df_normalized_reviews2.isna().any(axis=1)])
empty_or_nan_values = df_normalized_reviews2[df_normalized_reviews2.isna().any(axis=1)]
empty_or_nan_values


       rating  review normalized_review tokens filtered_tokens stemmed_text
59969       2  “ط. .                  ط  ['ط']              []          NaN
Empty DataFrame
Columns: [rating, review, normalized_review, tokens, filtered_tokens, stemmed_text]
Index: []


,rating,review,normalized_review,tokens,filtered_tokens,stemmed_text


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer # this is good: https://www.youtube.com/watch?v=rcovF6S1oFI
tfidf_vectorizer = TfidfVectorizer(
     # max_features=10000,      # Limit to 10k features # update i will use all and test chunking on all 71k unique words/features
    min_df=5,                # Remove rare words (appearing in less than 5 documents/rows)
    max_df=0.8,              # Remove overly common words (appearing in more than 80% of documents)
    # dtype='float32'          # Use float32 to save memory

)  #setting max features to non would basically run tf-idf on all 71030 unique words which is absurd if it crashes the memory ill test around 20k-50k will revisit pending looking at documentation and a few tutorials more
#so problem appears not to be with tf-idf itself but coneverting the sparse matrix into dense
#so no need to run with constraints just yet
# will try to do the conversion in a diff way but we need the dense matrix to use with tpot or other automl tools as matrix multiplication and ml models like rf use all the matrix (all values not just non zero values like in sparse)

# Fit and transform the stemmed text vocab
tfidf_matrix = tfidf_vectorizer.fit_transform(df_normalized_reviews2["stemmed_text"])

# Print the shape of the matrix
print("Shape of the TF-IDF matrix:", tfidf_matrix.shape)

# Convert sparse matrix to DataFrame
# tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Merge with the original DataFrame
# df_normalized_reviews = pd.concat([df_normalized_reviews, tfidf_df], axis=1)


# print(df_normalized_reviews.head())
# print(df_normalized_reviews.info())
#session crashes not enough rammmmmmmmmmmmmmmmm. i can run max of 7.5k or around that

Shape of the TF-IDF matrix: (102961, 9819)


#update using 71k words seems unprofessional might overfit and introduce noise and computation problems as im having them already.

#will probably use min_df and max_df to limit words more and try dense matrix again on drive first then maybe local but will always opt for drive till exhaustion

In [ ]:
print(h2o.cluster().status())


In [ ]:
!pip install h2o
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize H2O
h2o.init()

# Assuming 'df_normalized_reviews2' contains the 'rating' column and 'stemmed_text' column
df = df_normalized_reviews2[['rating', 'stemmed_text']]  # Selecting the 'rating' and 'stemmed_text' columns

# Convert the 'stemmed_text' column to a tf-idf sparse matrix
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['stemmed_text'])  # This is a sparse matrix

# Convert the sparse matrix to an H2OFrame (without converting to dense)
# H2O can handle sparse data structures efficiently
tfidf_h2o = h2o.H2OFrame(tfidf_matrix)

# Add the target column (rating) as a separate column in the H2OFrame
df_tfidf_h2o = tfidf_h2o.cbind(h2o.H2OFrame(df['rating'].values.tolist(), column_names=['rating']))

# Specify the target column and features
y = 'rating'
X = df_tfidf_h2o.columns
X.remove(y)  # Remove the target column 'rating' from features

# Split the data into training and testing sets (use H2O's split_frame method)
train, test = df_tfidf_h2o.split_frame(ratios=[.8], seed=1) #seed =1 to ensure that the random operations (like the train-test split or cross-validation splits) produce the same results each time the code is run so we can reproduce same results

#if you run the same code with the same seed, you’ll get the same results every time.


# Run AutoML with the training data
aml = H2OAutoML(max_models=10, seed=1, nfolds=5)  # You can adjust parameters as needed
aml.train(x=X, y=y, training_frame=train)

# View the leaderboard (top-performing models)
lb = aml.leaderboard
print(lb)

# Predict on the test set
preds = aml.predict(test)

# Evaluate the predictions
print(preds)

# Shutdown the H2O cluster
h2o.cluster().shutdown()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.9/265.9 MB 4.3 MB/s eta 0:00:00
Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.26" 2025-01-21; OpenJDK Runtime Environment (build 11.0.26+4-post-Ubuntu-1ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.26+4-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.11/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp1nn6ol1m
  JVM stdout: /tmp/tmp1nn6ol1m/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp1nn6ol1m/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,12 days
H2O_cluster_name:,H2O_from_python_unknownUser_44zbem
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
20:06:58.998: _train param, Dropping bad and constant columns: [C6628, C25720, C37708, C12408, C13739, C7952, C12406, C66276, C13734, C7954, C13732, C6625, C13730, C54287, C25721, C25722, C7969, C5309, C54299, C7963, C66265, C7964, C67596, C12415, C6632, C7962, C13744, C66260, C12410, C25716, C54297, C25715, C25712, C25713, C25742, C6607, C25740, C54269, C6609, C54268, C67586, C7931, C13757, C37728, C6604, C6603, C13750, C66256, C66257, C25749, C42287, C24419, C25747, C25731, C6619, C7947, C7948, C11109, C66241, C67573, C12438, C13768, C67572, C11104, C11105, C12436, C11100, C12431, C12432, C37710, C12430, C67578, C24407, C54273, C25732, C54270, C24430, C7915, C48402, C7918, C36419, C11118, C50723, C50721, C13775, C12442, C13774, C37740, C12440, C48409, C7927, C50730, C7926, C4841

In [ ]:
!nvidia-smi
